![](../docs/banner.png)

# Laboratorium 6: Produktyzacja procesów odkrywania wiedzy

<h2>Plan laboratorium<span class="tocSkip"></span></h2>
<hr>
<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#BHP">1. BHP</a></span></li>
        <li><span><a href="#Zasady-oceniania" data-toc-modified-id="2.-Zasady-oceniania-2">2. Zasady oceniania</a></span></li>
        <li><span><a href="#Środowiska-wirtualne" data-toc-modified-id="3.-Środowiska-wirtualne-3">3. Środowiska wirtualne</a></span></li>
        <li><span><a href="#Jupyter" data-toc-modified-id="4.-Jupyter-4">4. Środowisko Jupyter</a></span></li>
        <li><span><a href="#Git" data-toc-modified-id="5.-Git-5">5. Git i GitHub</a></span></li>
    </ul>
</div>

## Efekty kształcenia laboratorium
<hr>

- zapoznasz się z zasadami BHP
- dowiesz się, co będziemy ćwiczyć w trakcie tego semestru
- poznasz zasady oceniania 
- poznasz możliwości tworzenia środowiska wirtualnego Python
- poznasz środowisko Jupyter
- poznasz podstawowe mechanizmy systemu kontroli wersji Git i ekosystemu GitHub